In [1]:
import duckdb
import pandas as pd
MIMIC_DIR = "mimic-iii-clinical-database-1.4"

con = duckdb.connect("mimic_local.duckdb")


In [6]:
print("⚙️  Loading ICUSTAYS, PATIENTS, and ADMISSIONS...")

con.execute(f"""
CREATE OR REPLACE TABLE icustays AS
SELECT * FROM read_csv_auto('{MIMIC_DIR}/ICUSTAYS.csv.gz');
""")

con.execute(f"""
CREATE OR REPLACE TABLE patients AS
SELECT * FROM read_csv_auto('{MIMIC_DIR}/PATIENTS.csv.gz');
""")

con.execute(f"""
CREATE OR REPLACE TABLE admissions AS
SELECT * FROM read_csv_auto('{MIMIC_DIR}/ADMISSIONS.csv.gz');
""")
con.execute(f"""
CREATE OR REPLACE TABLE d_items AS
SELECT * FROM read_csv_auto('{MIMIC_DIR}/D_ITEMS.csv.gz');
""")
print("✅ Core tables loaded successfully.\n")

# --- Step 2: Load only vital-sign rows from CHARTEVENTS ---
print("⚙️  Loading CHARTEVENTS (vital signs subset only)... this may take 5–10 minutes.")
VITAL_ITEMIDS = [
    # Heart Rate
    211, 220045,
    # Systolic BP
    51, 442, 455, 220179, 220050,
    # Diastolic BP
    8368, 8440, 8502, 220180, 220051,
    # Respiratory Rate
    618, 220210,
    # Temperature
    223761, 676,
    # SpO₂
    646, 220277
]
itemid_list = ",".join(map(str, VITAL_ITEMIDS))
con.execute(f"""
CREATE OR REPLACE TABLE chartevents AS
SELECT *
FROM read_csv_auto('{MIMIC_DIR}/CHARTEVENTS.csv.gz',
                   all_varchar = true,
                   sample_size = -1,
                   ignore_errors = true)
WHERE TRY_CAST(itemid AS INTEGER) IN ({itemid_list});
""")
print("✅ CHARTEVENTS (vitals only) loaded successfully.")
print(con.execute("SELECT COUNT(*) AS n_rows FROM chartevents;").df())

⚙️  Loading ICUSTAYS, PATIENTS, and ADMISSIONS...
✅ Core tables loaded successfully.

⚙️  Loading CHARTEVENTS (vital signs subset only)... this may take 5–10 minutes.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ CHARTEVENTS (vitals only) loaded successfully.
     n_rows
0  31852450


In [13]:
# --- Step 3: Define adult cohort ---
print("\n⚙️  Filtering for adult cohort (18-89 years, ≥12 hours ICU stay)...")
def df_lower(query):
    df = con.execute(query).df()
    df.columns = df.columns.str.lower()
    return df

adult_cohort = df_lower("""
SELECT i.icustay_id,
       i.subject_id,
       i.hadm_id,
       i.intime,
       i.outtime,
       a.ethnicity,
       p.gender,
       (EXTRACT(YEAR FROM i.intime) - EXTRACT(YEAR FROM p.dob)) AS age,
       EXTRACT(EPOCH FROM (i.outtime - i.intime))/3600 AS los_hours
FROM icustays i
JOIN patients p USING (subject_id)
JOIN admissions a USING (hadm_id)
WHERE (EXTRACT(YEAR FROM i.intime) - EXTRACT(YEAR FROM p.dob)) BETWEEN 18 AND 89
  AND EXTRACT(EPOCH FROM (i.outtime - i.intime))/3600 >= 12;
""")
adult_ids = adult_cohort["icustay_id"].tolist()
print(f"✅ Found {len(adult_ids)} adult ICU stays (≥12 hours).\n")


⚙️  Filtering for adult cohort (18-89 years, ≥12 hours ICU stay)...
✅ Found 49689 adult ICU stays (≥12 hours).



In [15]:
# --- Step 4: Filter chartevents to adult cohort only ---
print("⚙️  Filtering CHARTEVENTS to adult cohort only...")
con.execute(f"""
CREATE OR REPLACE TABLE chartevents_adult AS
SELECT * FROM chartevents
WHERE TRY_CAST(icustay_id AS INTEGER) IN ({','.join(map(str, adult_ids))});
""")
print("✅ Filtered to adult ICU stays.")
print(con.execute("SELECT COUNT(*) AS n_rows FROM chartevents_adult;").df())


⚙️  Filtering CHARTEVENTS to adult cohort only...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Filtered to adult ICU stays.
     n_rows
0  28766989


In [16]:
# Convert data types and handle nulls
vitals_cleaned = df_lower("""
SELECT 
    TRY_CAST(icustay_id AS INTEGER) AS icustay_id,
    TRY_CAST(itemid AS INTEGER) AS itemid,
    charttime,
    TRY_CAST(valuenum AS DOUBLE) AS valuenum,
    valueuom
FROM chartevents_adult
WHERE valuenum IS NOT NULL 
  AND TRY_CAST(valuenum AS DOUBLE) IS NOT NULL
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
# Create a mapping for interpretability
vital_mapping = {
    'heart_rate': [211, 220045],
    'sbp': [51, 442, 455, 220179, 220050],
    'dbp': [8368, 8440, 8502, 220180, 220051],
    'resp_rate': [618, 220210],
    'temperature': [223761, 676],
    'spo2': [646, 220277]
}

# Add vital_sign column
def map_vital(itemid):
    for vital, ids in vital_mapping.items():
        if itemid in ids:
            return vital
    return 'unknown'

vitals_cleaned['vital_sign'] = vitals_cleaned['itemid'].apply(map_vital)

In [18]:
# Define reasonable ranges
ranges = {
    'heart_rate': (0, 300),
    'sbp': (0, 300),
    'dbp': (0, 200),
    'resp_rate': (0, 70),
    'temperature': (20, 45),  # Celsius
    'spo2': (0, 100)
}

# Filter outliers
def is_valid(row):
    vital = row['vital_sign']
    value = row['valuenum']
    if vital in ranges:
        min_val, max_val = ranges[vital]
        return min_val < value < max_val
    return True

vitals_cleaned = vitals_cleaned[vitals_cleaned.apply(is_valid, axis=1)]

In [19]:
# Join with intime to calculate time from admission
vitals_with_time = df_lower("""
SELECT 
    v.*,
    i.intime,
    EXTRACT(EPOCH FROM (v.charttime - i.intime))/3600 AS hours_from_admit
FROM vitals_cleaned v
JOIN icustays i USING (icustay_id)
""")

# Filter to first 24/48 hours if needed for early prediction
first_24h = vitals_with_time[vitals_with_time['hours_from_admit'].between(0, 24)]

BinderException: Binder Error: No function matches the given name and argument types '-(VARCHAR, TIMESTAMP)'. You might need to add explicit type casts.
	Candidate functions:
	-(TINYINT) -> TINYINT
	-(TINYINT, TINYINT) -> TINYINT
	-(SMALLINT) -> SMALLINT
	-(SMALLINT, SMALLINT) -> SMALLINT
	-(INTEGER) -> INTEGER
	-(INTEGER, INTEGER) -> INTEGER
	-(BIGINT) -> BIGINT
	-(BIGINT, BIGINT) -> BIGINT
	-(HUGEINT) -> HUGEINT
	-(HUGEINT, HUGEINT) -> HUGEINT
	-(FLOAT) -> FLOAT
	-(FLOAT, FLOAT) -> FLOAT
	-(DOUBLE) -> DOUBLE
	-(DOUBLE, DOUBLE) -> DOUBLE
	-(DECIMAL) -> DECIMAL
	-(DECIMAL, DECIMAL) -> DECIMAL
	-(UTINYINT) -> UTINYINT
	-(UTINYINT, UTINYINT) -> UTINYINT
	-(USMALLINT) -> USMALLINT
	-(USMALLINT, USMALLINT) -> USMALLINT
	-(UINTEGER) -> UINTEGER
	-(UINTEGER, UINTEGER) -> UINTEGER
	-(UBIGINT) -> UBIGINT
	-(UBIGINT, UBIGINT) -> UBIGINT
	-(UHUGEINT) -> UHUGEINT
	-(UHUGEINT, UHUGEINT) -> UHUGEINT
	-(BIGNUM) -> BIGNUM
	-(BIGNUM, BIGNUM) -> BIGNUM
	-(DATE, DATE) -> BIGINT
	-(DATE, INTEGER) -> DATE

In [ ]:
# Example: get summary statistics per ICU stay
vitals_summary = vitals_with_time.groupby(['icustay_id', 'vital_sign'])['valuenum'].agg([
    'mean', 'std', 'min', 'max', 'count'
]).reset_index()

# Pivot to wide format
vitals_wide = vitals_summary.pivot(
    index='icustay_id',
    columns='vital_sign',
    values=['mean', 'std', 'min', 'max']
)

In [22]:
con.execute(f"""
CREATE OR REPLACE TABLE d_items AS
SELECT * FROM read_csv_auto('{MIMIC_DIR}/D_ITEMS.csv.gz');
""")


In [62]:
vent_ids = con.execute("""
WITH candidates AS (
  SELECT
    CAST(itemid AS INTEGER) AS itemid,
    lower(coalesce(label, ''))    AS lab,
    lower(coalesce(category, '')) AS cat
  FROM d_items
),
hits AS (
  SELECT DISTINCT itemid
  FROM candidates
  WHERE
    lab LIKE '%vent%' OR cat LIKE '%vent%'
    OR lab LIKE '%mechanical vent%' OR lab LIKE '%ventilator mode%'
    OR lab LIKE '%intubat%' OR lab LIKE '%extubat%'
    OR lab LIKE '%peep%' OR lab LIKE '%psv%'
    OR lab LIKE '%pip%' OR lab LIKE '%plateau%'
    OR lab LIKE '%tidal vol%' OR lab LIKE '%vt%'
    OR lab LIKE '%fio2%' OR lab LIKE '%inspired o2%'
    OR lab LIKE '%minute vent%' OR lab LIKE '%rr set%'
)
SELECT itemid
FROM hits
ORDER BY itemid;
""").df()


vent_ids.columns = [c.strip().lower() for c in vent_ids.columns]

VENT_ITEMIDS = vent_ids["itemid"].astype(int).tolist()
ids_csv = ",".join(map(str, VENT_ITEMIDS))
print(f"{len(VENT_ITEMIDS)} vent-related ITEMIDs collected.")
ids_csv.to_csv("old_vent_itemids.csv", index=False)


313 vent-related ITEMIDs collected.


AttributeError: 'str' object has no attribute 'to_csv'

In [63]:
vent_labels = con.execute(f"""
SELECT itemid, label
FROM d_items
WHERE itemid IN ({ids_csv})
ORDER BY itemid;
""").df()

vent_labels.to_csv("vent_itemids.csv", index=False)
print(f"Saved {len(vent_labels)}  vent-related items to vent_itemids_labels_filtered.csv")

all_exclusions = (
    # Generic/False Positive Filter (Parentheses escaped)
    "intervent|ventric|combivent|flovent|piperacillin|atrovent|prevent|event|advent|conven|"
    "esdep|floor|or sent|or received|defibrillation|pneumothorax|operation|fluoroscopy|nuclear|fall|brain death|timeout|"
    "hand cleansing|barrier precautions|no complications|"
    "attending|supervisor|checklist|gcsverbal|apache|patient identified|identified correctly|"
    "transferred|transfer|temporary ventricular|temporary pacemaker|pacer wires|stim thresh|sens thresh|"
    "vent drain|ventric|cpp|icp|cap|flowvent|chest pain|cardiac arrest|respiratory arrest|chest opened|line/catheter removal|peritoneal fluid|c-spine|tls clearance|"
    
    # Airway Management Procedures (DROP)
    "intubat|extubat|ett|tube secured|stylette|bougie|larynx|laryng|mask vent|rsi|"
    "pre-oxygen|difficult to intubat|induction|drugs|thyromental|mandibular|dentition|neck rom|airway assessment|"
    "propofol|ketamine|etomidate|vecuronium|cis-atracurium|succ|pre-oxy|blade|equipment|"
    
    # Measurements/Observed Values (DROP: Note the proper escaping of parentheses)
    "measured|observ|spont|tidal volume \\(ml\\)|vt \\(ml\\)|volume measured|"
    "peep observed|p-peak observed|pi observed|pmean observed|p mean observed|"
    "etco2|p_exp|exp flow|exp volume|exp vt|"
    "insp flow|inspiratory flow|p insp|p_insp|flow measured|tidal volume measured|"
    "vent rate observed|resp rate observed|min vent measured|minute vol observed|flow \\(l/s\\)|" # Flow (l/s) escaped
    
    # Alarm Settings (DROP)
    "alarm|low|high|apnea|apnoea|"
    
    # Secondary/Derived Values (DROP)
    "ratio|i:e ratio|vd/vt|ie ratio|compliance|resistance|trigger|trigger flow"
)

# Apply the filter with regex=True to utilize the '|' operator correctly
# and case=False for case-insensitive matching.
vent_labels_filtered = vent_labels[
    ~vent_labels["LABEL"].str.contains(
        all_exclusions,
        case=False,
        na=False,
        regex=True # Confirms the string is a regex, resolves the warning
    )
]

vent_labels_filtered.to_csv("vent_itemids_labels_filtered.csv", index=False)
print(f"Saved {len(vent_labels_filtered)} filtered vent-related items to vent_itemids_labels_filtered.csv")

Saved 313  vent-related items to vent_itemids_labels_filtered.csv
Saved 98 filtered vent-related items to vent_itemids_labels_filtered.csv


In [7]:
import pandas as pd

# Define the file path and the target column name
file_path = "new_snap_lookback12_horizon12.csv"

column_name = "y_vent_in_horizon"

try:
    # 1. Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # 2. Check if the column exists
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file.")
        print("Available columns are:", list(df.columns))
    else:
        # 3. Count the number of times the value '1' appears in the specified column
        # .value_counts() returns a series of counts for unique values.
        # .get(1, 0) retrieves the count for the value 1, or 0 if 1 is not present.
        count_of_ones = df[column_name].value_counts().get(1, 0)
        
        # 4. Print the result
        print(f"Successfully loaded file: {file_path}")
        print(f"The number of '1's in the column '{column_name}' is: {count_of_ones}")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please ensure the file is in the correct directory or update the 'file_path'.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully loaded file: new_snap_lookback12_horizon12.csv
The number of '1's in the column 'y_vent_in_horizon' is: 26089
